In [2]:
!pip install pandas numpy matplotlib seaborn scikit-learn
!pip install pytorch-lightning
!pip install torch
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.3/857.3 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.9/413.9 kB 24.9 MB/s eta 0:00:00


In [3]:
# =====================================================
# 1. IMPORT LIBRARIES
# =====================================================

import pandas as pd
import numpy as np
import optuna

from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [4]:
# =====================================================
# 2. LOAD DATA
# =====================================================

historical = pd.read_csv("/content/drive/MyDrive/Skripsi/Dataset/Wind Power/wind farm historical data.csv")
nwp = pd.read_csv("/content/drive/MyDrive/Skripsi/Dataset/Wind Power/NWP.csv")

historical.columns = historical.columns.str.strip()
nwp.columns = nwp.columns.str.strip()

# Convert datetime (different formats)
historical['Date'] = pd.to_datetime(historical['Date'], dayfirst=True)
nwp['time'] = pd.to_datetime(nwp['time'], format='mixed')

# Rename columns
historical = historical.rename(columns={
    'Date': 'timestamp',
    'Speed': 'wind_speed',
    'Direction': 'wind_direction',
    'Energy': 'power'
})

nwp = nwp.rename(columns={
    'time': 'timestamp',
    'mod': 'wind_speed_nwp',
    'dir': 'wind_dir_nwp',
    'temp': 'temperature_nwp',
    'rh': 'humidity_nwp',
    'mslp': 'pressure_nwp'
})

# Merge
data = pd.merge(historical, nwp, on='timestamp', how='inner')
data = data.sort_values('timestamp').reset_index(drop=True)
data = data.ffill().dropna()

print("Merged data shape:", data.shape)

Merged data shape: (8784, 9)


In [5]:
# =====================================================
# 3. FEATURE ENGINEERING
# =====================================================

data_fe = data.copy()

# Time features
data_fe['hour'] = data_fe['timestamp'].dt.hour
data_fe['month'] = data_fe['timestamp'].dt.month

data_fe['hour_sin'] = np.sin(2*np.pi*data_fe['hour']/24)
data_fe['hour_cos'] = np.cos(2*np.pi*data_fe['hour']/24)
data_fe['month_sin'] = np.sin(2*np.pi*data_fe['month']/12)
data_fe['month_cos'] = np.cos(2*np.pi*data_fe['month']/12)

# Lag features
for lag in [1,2,3,6,12,24]:
    data_fe[f'power_lag_{lag}'] = data_fe['power'].shift(lag)

# Rolling stats
data_fe['rolling_mean_6'] = data_fe['power'].rolling(6).mean()
data_fe['rolling_std_6'] = data_fe['power'].rolling(6).std()

# Physics-inspired
data_fe['wind_speed_cubed'] = data_fe['wind_speed']**3
data_fe['wind_speed_nwp_cubed'] = data_fe['wind_speed_nwp']**3

# Regime classification
def classify_regime(ws):
    if ws < 3:
        return 0
    elif ws < 12:
        return 1
    else:
        return 2

data_fe['regime'] = data_fe['wind_speed'].apply(classify_regime)

data_fe = data_fe.dropna().reset_index(drop=True)

print("After feature engineering:", data_fe.shape)

After feature engineering: (8760, 26)


In [6]:
# =====================================================
# 4. TIME-BASED SPLIT
# =====================================================

train_size = int(len(data_fe)*0.7)
val_size = int(len(data_fe)*0.15)

train = data_fe[:train_size]
val = data_fe[train_size:train_size+val_size]
test = data_fe[train_size+val_size:]

features = data_fe.drop(['timestamp','power'], axis=1).columns

X_train = train[features]
y_train = train['power']

X_val = val[features]
y_val = val['power']

X_test = test[features]
y_test = test['power']

# Scaling
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [7]:
# =====================================================
# 5. OPTUNA HYPERPARAMETER TUNING
# =====================================================

def objective(trial):

    params = {
        'n_estimators': trial.suggest_int('n_estimators', 400, 1500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 5),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 5),
        'random_state': 42,
        'n_jobs': -1
    }

    model = XGBRegressor(**params)
    model.fit(X_train_scaled, y_train)

    pred_val = model.predict(X_val_scaled)
    rmse = np.sqrt(mean_squared_error(y_val, pred_val))
    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

print("Best parameters:", study.best_params)

[I 2026-02-18 09:48:07,382] A new study created in memory with name: no-name-4ad18096-d1e6-4293-a8d0-9058ec0714f9
[I 2026-02-18 09:48:14,174] Trial 0 finished with value: 100.19118409968604 and parameters: {'n_estimators': 1150, 'max_depth': 10, 'learning_rate': 0.14264470449550434, 'subsample': 0.9742423115547953, 'colsample_bytree': 0.6738706747081078, 'gamma': 4.9684516637906455, 'min_child_weight': 10, 'reg_alpha': 4.836811733625657, 'reg_lambda': 2.2946627997677593}. Best is trial 0 with value: 100.19118409968604.
[I 2026-02-18 09:48:15,198] Trial 1 finished with value: 110.42196809040512 and parameters: {'n_estimators': 925, 'max_depth': 3, 'learning_rate': 0.011716039394253554, 'subsample': 0.7077162478770891, 'colsample_bytree': 0.6467217813799979, 'gamma': 0.46763079426061227, 'min_child_weight': 1, 'reg_alpha': 4.837664958922753, 'reg_lambda': 1.5156444000883462}. Best is trial 0 with value: 100.19118409968604.
[I 2026-02-18 09:48:16,137] Trial 2 finished with value: 99.90700

Best parameters: {'n_estimators': 1431, 'max_depth': 7, 'learning_rate': 0.056544312319452575, 'subsample': 0.7795063938755233, 'colsample_bytree': 0.675630595052698, 'gamma': 1.6064077072540024, 'min_child_weight': 4, 'reg_alpha': 4.5111850398854525, 'reg_lambda': 3.2480622295264516}


In [8]:
# =====================================================
# 6. GLOBAL MODEL (OPTIMIZED)
# =====================================================

best_model = XGBRegressor(**study.best_params)
best_model.fit(X_train_scaled, y_train)

pred_global = best_model.predict(X_test_scaled)

rmse_global = np.sqrt(mean_squared_error(y_test, pred_global))
r2_global = r2_score(y_test, pred_global)

print("\nGlobal Optimized Model")
print("RMSE:", rmse_global)
print("R2:", r2_global)


Global Optimized Model
RMSE: 146.5480094732497
R2: 0.9282538217455112


In [9]:
# =====================================================
# 7. REGIME-BASED MODELING
# =====================================================

models = {}
scalers = {}

for regime_id in sorted(train['regime'].unique()):

    train_r = train[train['regime']==regime_id]

    if len(train_r) < 100:
        continue

    X_r = train_r[features]
    y_r = train_r['power']

    scaler_r = MinMaxScaler()
    X_r_scaled = scaler_r.fit_transform(X_r)

    model_r = XGBRegressor(**study.best_params)
    model_r.fit(X_r_scaled, y_r)

    models[regime_id] = model_r
    scalers[regime_id] = scaler_r


# Prediction per regime
predictions = []
true_values = []

for regime_id in models.keys():

    test_r = test[test['regime']==regime_id]

    if len(test_r)==0:
        continue

    X_r_test = scalers[regime_id].transform(test_r[features])
    preds = models[regime_id].predict(X_r_test)

    predictions.extend(preds)
    true_values.extend(test_r['power'])

predictions = np.array(predictions)
true_values = np.array(true_values)

rmse_regime = np.sqrt(mean_squared_error(true_values, predictions))
r2_regime = r2_score(true_values, predictions)

print("\nRegime-Based Model")
print("RMSE:", rmse_regime)
print("R2:", r2_regime)


Regime-Based Model
RMSE: 150.35874345973062
R2: 0.9254258676245378
